# Phân Tích Dữ Liệu Khách Hàng QVI

Notebook này thực hiện phân tích chi tiết dữ liệu giao dịch và hành vi mua hàng của khách hàng QVI.

## 1. Thiết Lập Ban Đầu và Tải Thư Viện

Trước tiên, chúng ta cần tải các thư viện cần thiết và thiết lập một số cấu hình mặc định cho biểu đồ.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # Added import for regular expressions

# Set style for plots
# Changed 'seaborn' to a current style like 'seaborn-v0_8-whitegrid'
plt.style.use('seaborn-v0_8-whitegrid') 
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = [12, 6]

## 2. Định Nghĩa Các Hàm Xử Lý

### 2.1. Tải Dữ Liệu

In [ ]:
def load_data():
    """Load the datasets"""
    print("Loading data...")
    # Ensure these files are in the same directory as your script,
    # or provide the full path to them.
    try:
        transaction_data = pd.read_excel('QVI_transaction_data.xlsx')
        purchase_behavior = pd.read_csv('QVI_purchase_behaviour.csv')
        print("Data loaded successfully.")
        return transaction_data, purchase_behavior
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        print("Please ensure 'QVI_transaction_data.xlsx' and 'QVI_purchase_behaviour.csv' are in the correct directory.")
        return None, None

### 2.2. Kiểm Tra Dữ Liệu Ban Đầu

In [ ]:
def examine_data(transaction_data, purchase_behavior):
    """Examine the structure of the data"""
    if transaction_data is None or purchase_behavior is None:
        print("Cannot examine data as it was not loaded.")
        return

    print("\nTransaction Data Info:")
    transaction_data.info()
    print("\nSample of transaction data:")
    print(transaction_data.head())
    
    print("\nPurchase Behavior Info:")
    purchase_behavior.info()
    print("\nSample of purchase behavior data:")
    print(purchase_behavior.head())

### 2.3. Làm Sạch Dữ Liệu Giao Dịch

In [ ]:
def clean_transaction_data(df):
    """Clean and preprocess transaction data"""
    if df is None:
        print("Transaction data is None, skipping cleaning.")
        return None
    print("\nCleaning transaction data...")
    
    # Convert DATE to datetime
    # It's good practice to handle potential errors during conversion
    try:
        df['DATE'] = pd.to_datetime(df['DATE'], origin='1899-12-30', unit='D')
    except Exception as e:
        print(f"Error converting DATE column to datetime: {e}")
        return df 
    
    print("\nExamining product names:")
    print(df['PROD_NAME'].value_counts().head())
    
    if not df['PROD_NAME'].empty and df['PROD_NAME'].dtype == 'object':
        try:
            words = pd.Series(' '.join(df['PROD_NAME'].dropna()).split()).value_counts()
            words = words[~words.index.str.contains('\\d', na=False)] 
            words = words[words.index.str.match('^[a-zA-Z]+$', na=False)] 
            print("\nMost common words in product names (alphabetic only):")
            print(words.head(10))
        except Exception as e:
            print(f"Error analyzing words in product names: {e}")
    else:
        print("PROD_NAME column is empty or not of string type, skipping word analysis.")

    df = df[~df['PROD_NAME'].str.contains('salsa', case=False, na=False)]
    
    print("\nChecking for outliers in product quantity:")
    print(df['PROD_QTY'].describe())
    
    high_qty = df[df['PROD_QTY'] == 200]
    print("\nTransactions with quantity = 200:")
    print(high_qty)
    
    if 'LYLTY_CARD_NBR' in df.columns:
        df = df[df['LYLTY_CARD_NBR'] != 226000]
    else:
        print("LYLTY_CARD_NBR column not found, skipping outlier customer removal.")
        
    print("Transaction data cleaning complete.")
    return df

### 2.4. Phân Tích Ngày Giao Dịch

In [ ]:
def analyze_dates(df):
    """Analyze transaction dates"""
    if df is None or 'DATE' not in df.columns:
        print("Data or DATE column is missing, skipping date analysis.")
        return
    print("\nAnalyzing transaction dates...")
    
    daily_transactions = df.groupby('DATE').size()
    
    if daily_transactions.empty:
        print("No transactions found to analyze by date.")
        return

    plt.figure()
    daily_transactions.plot()
    plt.title('Transactions over time')
    plt.xlabel('Date')
    plt.ylabel('Number of transactions')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    december_transactions = daily_transactions[daily_transactions.index.month == 12]
    if not december_transactions.empty:
        plt.figure()
        december_transactions.plot()
        plt.title('December Transactions')
        plt.xlabel('Date')
        plt.ylabel('Number of transactions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No transactions found in December.")
    print("Date analysis complete.")

### 2.5. Phân Tích Kích Thước Gói Sản Phẩm

In [ ]:
def analyze_pack_sizes(df):
    """Analyze pack sizes"""
    if df is None or 'PROD_NAME' not in df.columns:
        print("Data or PROD_NAME column is missing, skipping pack size analysis.")
        return df
    print("\nAnalyzing pack sizes...")
    
    def extract_pack_size(name):
        if pd.isna(name):
            return None
        match = re.search(r'(\d+)g', str(name)) 
        if match:
            return int(match.group(1))
        return None

    df['PACK_SIZE'] = df['PROD_NAME'].apply(extract_pack_size)
    
    if 'PACK_SIZE' in df.columns and not df['PACK_SIZE'].dropna().empty:
        pack_size_dist = df['PACK_SIZE'].value_counts().sort_index()
        print("\nPack size distribution:")
        print(pack_size_dist)
        
        plt.figure()
        plt.hist(df['PACK_SIZE'].dropna(), bins=30) 
        plt.title('Distribution of Pack Sizes')
        plt.xlabel('Pack Size (g)')
        plt.ylabel('Frequency')
        plt.show()
    else:
        print("No pack sizes found or PACK_SIZE column is empty.")
    print("Pack size analysis complete.")
    return df

### 2.6. Phân Tích Thương Hiệu

In [ ]:
def analyze_brands(df):
    """Analyze brands"""
    if df is None or 'PROD_NAME' not in df.columns:
        print("Data or PROD_NAME column is missing, skipping brand analysis.")
        return df
    print("\nAnalyzing brands...")
    
    df['BRAND'] = df['PROD_NAME'].apply(lambda x: str(x).split()[0].upper() if pd.notna(x) and str(x).split() else None)
    
    brand_mapping = {
        'RRD': 'RED_ROCK_DELI',
        'SNBTS': 'SUNBITES',
        'INFZNS': 'INFUZIONS',
        'WW': 'WOOLWORTHS',
        'SMITH': 'SMITHS',
        'NCC': 'NATURAL',
        'DORITO': 'DORITOS',
        'GRAIN': 'GRAINWAVES'
    }
    df['BRAND'] = df['BRAND'].replace(brand_mapping)
    
    if 'BRAND' in df.columns and not df['BRAND'].dropna().empty:
        print("\nBrand distribution:")
        print(df['BRAND'].value_counts())
    else:
        print("No brands found or BRAND column is empty.")
    print("Brand analysis complete.")
    return df

### 2.7. Phân Tích Phân Khúc Khách Hàng

In [ ]:
def analyze_customer_segments(data):
    """Analyze customer segments"""
    if data is None or not all(col in data.columns for col in ['TOT_SALES', 'PREMIUM_CUSTOMER', 'LIFESTAGE', 'LYLTY_CARD_NBR']):
        print("Data or required columns (TOT_SALES, PREMIUM_CUSTOMER, LIFESTAGE, LYLTY_CARD_NBR) are missing, skipping customer segment analysis.")
        return
    print("\nAnalyzing customer segments...")
    
    try:
        sales = data.pivot_table(
            values='TOT_SALES',
            index='PREMIUM_CUSTOMER',
            columns='LIFESTAGE',
            aggfunc='sum'
        )
        
        if sales.empty:
            print("Pivot table for sales is empty.")
        else:
            sales_prop = sales.div(sales.sum().sum()) * 100
            plt.figure()
            sales_prop.plot(kind='bar', stacked=True)
            plt.title('Proportion of Sales by Customer Segment')
            plt.xlabel('Premium Customer Flag')
            plt.ylabel('Proportion of Sales (%)')
            plt.legend(title='Lifestage', bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Error creating or plotting sales pivot table: {e}")

    try:
        customer_counts = data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['LYLTY_CARD_NBR'].nunique()
        print("\nCustomer counts by segment:")
        print(customer_counts)
    except Exception as e:
        print(f"Error calculating customer counts: {e}")
    print("Customer segment analysis complete.")

### 2.8. Phân Tích Hành Vi Mua Hàng

In [ ]:
def analyze_purchase_behavior(data):
    """Analyze purchase behavior"""
    if data is None or not all(col in data.columns for col in ['LIFESTAGE', 'PREMIUM_CUSTOMER', 'PROD_QTY', 'TOT_SALES', 'LYLTY_CARD_NBR']):
        print("Data or required columns are missing, skipping purchase behavior analysis.")
        return
    print("\nAnalyzing purchase behavior...")
    
    try:
        if data['LYLTY_CARD_NBR'].nunique() > 0:
            avg_units = data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER']).agg(
                AVG_UNITS_PER_CUSTOMER_OVERALL = ('PROD_QTY', lambda x: x.sum() / data['LYLTY_CARD_NBR'].nunique())
            ).round(2)
            print("\nAverage units (based on overall unique customers):")
            print(avg_units)
        else:
            print("No unique customers found to calculate average units.")
    except Exception as e:
        print(f"Error calculating average units: {e}")

    data['PRICE_PER_UNIT'] = data.apply(lambda row: row['TOT_SALES'] / row['PROD_QTY'] if row['PROD_QTY'] != 0 else 0, axis=1)
    
    try:
        avg_price = data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['PRICE_PER_UNIT'].mean().round(2)
        print("\nAverage price per unit:")
        print(avg_price)
        
        if not avg_price.empty:
            avg_price_pivot = avg_price.unstack()
            plt.figure()
            avg_price_pivot.plot(kind='bar')
            plt.title('Average Price per Unit by Segment')
            plt.xlabel('Lifestage') 
            plt.ylabel('Average Price ($)')
            plt.legend(title='Premium Customer', bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Error calculating or plotting average price: {e}")
    print("Purchase behavior analysis complete.")

### 2.9. Phân Tích Mức Độ Ưa Thích Thương Hiệu (Brand Affinity)

In [ ]:
def analyze_brand_affinity(data, target_segment):
    """Analyze brand affinity for target segment"""
    if data is None or 'BRAND' not in data.columns or 'LIFESTAGE' not in data.columns:
        print("Data, BRAND, or LIFESTAGE column is missing, skipping brand affinity analysis.")
        return
    print(f"\nAnalyzing brand affinity for {target_segment}...")
    
    target = data[data['LIFESTAGE'] == target_segment]
    others = data[data['LIFESTAGE'] != target_segment]
    
    if target.empty:
        print(f"No data found for target segment: {target_segment}")
        return
    if others.empty:
        print("No data found for other segments to compare.")
        return

    target_props = target['BRAND'].value_counts(normalize=True)
    other_props = others['BRAND'].value_counts(normalize=True)
    
    affinity = pd.Series(dtype=float)
    for brand, prop in target_props.items():
        if brand in other_props and other_props[brand] > 0:
            affinity[brand] = prop / other_props[brand]
        else:
            affinity[brand] = float('inf') 

    affinity = affinity.sort_values(ascending=False).round(2)
    
    print(f"\nBrand affinity scores (>1 means {target_segment} more likely to purchase):")
    print(affinity.head(20)) 
    
    if not affinity.empty:
        plt.figure(figsize=(14, 7)) 
        affinity.head(10).plot(kind='bar') 
        plt.title(f'Top 10 Brand Affinity for {target_segment}')
        plt.xlabel('Brand')
        plt.ylabel('Affinity Score (Higher is better)')
        plt.axhline(y=1, color='grey', linestyle='--') 
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    print("Brand affinity analysis complete.")

## 3. Thực Hiện Phân Tích

### 3.1. Tải Dữ Liệu

In [ ]:
transaction_data, purchase_behavior = load_data()

# Kiểm tra xem dữ liệu có được tải thành công không trước khi tiếp tục
if transaction_data is None or purchase_behavior is None:
    print("Exiting due to data loading failure.")
    # Trong Jupyter Notebook, bạn có thể dừng ở đây hoặc xử lý lỗi theo cách khác
else:
    print("Data successfully loaded and ready for examination.")

### 3.2. Kiểm Tra Dữ Liệu Ban Đầu

In [ ]:
if transaction_data is not None and purchase_behavior is not None:
    examine_data(transaction_data, purchase_behavior)

### 3.3. Làm Sạch Dữ Liệu Giao Dịch

In [ ]:
if transaction_data is not None:
    transaction_data_cleaned = clean_transaction_data(transaction_data.copy()) # Làm việc trên bản sao để giữ dữ liệu gốc
    if transaction_data_cleaned is None:
        print("Exiting due to transaction data cleaning failure.")

### 3.4. Phân Tích Ngày Giao Dịch

In [ ]:
if 'transaction_data_cleaned' in locals() and transaction_data_cleaned is not None:
    analyze_dates(transaction_data_cleaned)

### 3.5. Phân Tích Kích Thước Gói Sản Phẩm

In [ ]:
if 'transaction_data_cleaned' in locals() and transaction_data_cleaned is not None:
    transaction_data_analyzed_pack = analyze_pack_sizes(transaction_data_cleaned.copy())

### 3.6. Phân Tích Thương Hiệu

In [ ]:
if 'transaction_data_analyzed_pack' in locals() and transaction_data_analyzed_pack is not None:
    transaction_data_analyzed_brands = analyze_brands(transaction_data_analyzed_pack.copy())

### 3.7. Gộp Dữ Liệu Giao Dịch và Hành Vi Mua Hàng

In [ ]:
data_merged = None
if ('transaction_data_analyzed_brands' in locals() and transaction_data_analyzed_brands is not None and 
    purchase_behavior is not None):
    
    if 'LYLTY_CARD_NBR' not in transaction_data_analyzed_brands.columns or \
       'LYLTY_CARD_NBR' not in purchase_behavior.columns:
        print("LYLTY_CARD_NBR column missing in one or both dataframes. Cannot merge.")
    else:
        data_merged = pd.merge(transaction_data_analyzed_brands, purchase_behavior, on='LYLTY_CARD_NBR', how='left')
        print("\nData merged successfully.")
        print(f"Shape of merged data: {data_merged.shape}")
        print(data_merged.head())
else:
    print("Skipping merge due to missing prerequisite data.")

if data_merged is not None and data_merged.empty:
    print("Merged data is empty. Stopping further analysis.")

### 3.8. Phân Tích Phân Khúc Khách Hàng

In [ ]:
if data_merged is not None and not data_merged.empty:
    analyze_customer_segments(data_merged)

### 3.9. Phân Tích Hành Vi Mua Hàng

In [ ]:
if data_merged is not None and not data_merged.empty:
    analyze_purchase_behavior(data_merged)

### 3.10. Phân Tích Mức Độ Ưa Thích Thương Hiệu

In [ ]:
if data_merged is not None and not data_merged.empty and 'LIFESTAGE' in data_merged.columns:
    target_segment_young = 'YOUNG SINGLES/COUPLES'
    if target_segment_young in data_merged['LIFESTAGE'].unique():
        analyze_brand_affinity(data_merged, target_segment_young)
    else:
        print(f"\nTarget segment '{target_segment_young}' not found in LIFESTAGE. Available segments:")
        print(data_merged['LIFESTAGE'].unique())
    
    target_segment_retirees = 'RETIREES'
    if target_segment_retirees in data_merged['LIFESTAGE'].unique():
         analyze_brand_affinity(data_merged, target_segment_retirees)
    else:
        print(f"\nTarget segment '{target_segment_retirees}' not found in LIFESTAGE.")

## 4. Lưu Kết Quả

In [ ]:
if data_merged is not None and not data_merged.empty:
    try:
        data_merged.to_csv('processed_data_qvi.csv', index=False)
        print("\nAnalysis complete! Processed data saved to 'processed_data_qvi.csv'")
    except Exception as e:
        print(f"Error saving processed data: {e}")
else:
    print("\nNo processed data to save.")

--- Kết thúc phân tích ---